In [67]:
from datetime import datetime
from typing import Dict, Union, List, Optional, Any

from pydantic import BaseModel, PositiveInt, EmailStr, computed_field, field_validator, HttpUrl, model_validator, ValidationError

In [72]:
class User(BaseModel):
    #fields of the models are defined as class variables

    name: str #class variable for user name of type string
    email: str #class variable for user email of type string
    account_id: int #class variable for user account of type int

In [73]:
user = User( #if the data passed to the User class is valid, the user object
            #will be successfully created
    name='Maaz',
    email='mmaazkhan@outlook.com',
    account_id=21
)
print(user)
print(user.email)

name='Maaz' email='mmaazkhan@outlook.com' account_id=21
mmaazkhan@outlook.com


In [74]:
user_data = { #dictionary variable containing all user data
    'name': 'Khan',
    'email': 'khan@outlook.com',
    'account_id': 31
}

user = User(**user_data) #unpack the user data
print(user)

name='Khan' email='khan@outlook.com' account_id=31


In [75]:
#if data passed is not valid error will be raised

#user = User(
#    name='Maaz',
#    email='maaz@outlook.com',
#    account_id = 'hello'
#)
#print(user)

#ValidationError: account_id input should be a valid integer, unable to 
#parse string as an integer

In [81]:
#However, if you pass a number in string format, it will parse it into an integer.
#However, the IDE will underline the property indicating error

user = User(
    name='Maaz',
    email='maaz@outlook.com',
    account_id = '21' #IDE still gives an error
)

print(user) #the code will run and the account_id will be parse to integer
print(type(user.account_id)) #type will int

name='Maaz' email='maaz@outlook.com' account_id=21
<class 'int'>


In [84]:
#However, we can enforce the model to not parse it (strict mode)
try:
    User.model_validate({'name': 'maaz', 'email':'mmaazkhan@mail.com','account_id':'21'}, strict=True)
    #it will not parse the account_id to integer.
except ValidationError as error:
    print(error)

1 validation error for User
account_id
  Input should be a valid integer [type=int_type, input_value='21', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/int_type


In [7]:
# we can also ensure that the email entered by the user is a valid email

user = User(name='Maaz', email='my email', account_id=1)
print(user.email) #the email is not a valid email address


my email


In [8]:
class User(BaseModel):
    name: str
    email: EmailStr
    account_id: int
    
user = User(name='Maaz', email='maaz@outlook.com', account_id=5)
print(user.email) #the email is now valid email address
print(type(user.email))

#user = User(name='Maaz', email='maaz', account_id=5) 
#this will raise validation error stating the email value is not a valid email address

maaz@outlook.com
<class 'str'>


In [15]:
# we can also add custom validation for our model
class User(BaseModel):
    name: str
    email: EmailStr
    account_id: int

    @field_validator('account_id')
    def validate_account_id(cls, value): # we have added custom validation that the
        #account number must be positive integer (its value must be greater than 0)
        if value <=0:
            raise ValueError(f"account_id must be positive: {value}")
        return value

In [17]:
user = User(name='Allen', email='allen@mail.com', account_id=25) #works fine
print(user)

#user = User(name='Allen', email='allen@mail.com', account_id=-56)
#will give validation error that the account number should be positive

name='Allen' email='allen@mail.com' account_id=25


In [30]:
#we can covert the user data to a json format
user_json_data = user.model_dump_json() #json has been deprecated. use model_dump_json 
print(user_json_data)
print(type(user_json_data))

{"name":"Allen","email":"allen@mail.com","account_id":25}
<class 'str'>


In [24]:
#you can convert json data back to pydantic model

user = User.model_validate_json(user_json_data) #parse_raw has been deprecated. use model_validate_json
print(user)


name='Allen' email='allen@mail.com' account_id=25


In [31]:
# you can convert the model data into dictionary
print(user.model_dump())
print(type(user.model_dump()))

{'name': 'Allen', 'email': 'allen@mail.com', 'account_id': 25}
<class 'dict'>


In [34]:
#we can generate a JSON schema for a model class
display(user.model_json_schema())

{'properties': {'name': {'title': 'Name', 'type': 'string'},
  'email': {'format': 'email', 'title': 'Email', 'type': 'string'},
  'account_id': {'title': 'Account Id', 'type': 'integer'}},
 'required': ['name', 'email', 'account_id'],
 'title': 'User',
 'type': 'object'}

In [54]:
class User(BaseModel):
    name: str
    email: EmailStr
    age: PositiveInt #age should be positive
    profileUrl: HttpUrl
    
user = User(
    name='Maaz', 
    email='mmaazkhan@mail.com', 
    age=25,
    profileUrl='https://mmaazkhanhere.com'
    #profileUrl = 'hello'  this will raise error
    ) #no error
#user = User(name='Maaz', email='mmaazkhan@mail.com', age=-22) 
#this will raise value error as it expects age to be positive number
print(user)

name='Maaz' email='mmaazkhan@mail.com' age=25 profileUrl=Url('https://mmaazkhanhere.com/')


In [66]:
#entire model can be validated using @model_validator

class Owner(BaseModel):
    name: str
    email: EmailStr
    
    @model_validator(mode='before')
    @classmethod
    def check_sensitive_info_omitted(cls, data: Any)-> Any:
        if isinstance(data, dict):
            if 'password' in data:
                raise ValueError('Password should not be included')
            if 'card_number' in data:
                raise ValueError('Card number should not be included')
        return data
    
    @model_validator(mode='after')
    def check_name_contain_space(self)-> 'Owner':
        if ' ' not in self.name:
            raise ValueError('Owner name must contain a space')
        return self
    
print(Owner(name='John Doe', email='john@example.com'))

try:
    print(Owner(name='Maaz Khan', email='john.doe@example.com', card_number = 22))
    
except ValidationError as e:
    print(e)

name='John Doe' email='john@example.com'
1 validation error for Owner
  Value error, Card number should not be included [type=value_error, input_value={'name': 'Maaz Khan', 'em...com', 'card_number': 22}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error


In [70]:
#computed fields is used to calculate a value dynamically

class Person(BaseModel):
    name: str
    birth_year: int
    
    @computed_field
    @property
    def age(self)-> int:
        current_year = datetime.now().year
        return current_year - self.birth_year
    
print(Person(name='John Doe', birth_year=1998))

name='John Doe' birth_year=1998 age=26


### Nested Models


In [48]:
class Food(BaseModel): # a class Food that inherits Base Model from pydantic
    name: str #name of the food
    price: float #price of the food
    ingredients: Optional[List[str]] = None #optional parameter with default value of None
    
class Restaurant(BaseModel): # a restaurant class
    name: str #name of the restaurant
    location: str #location of the restaurant
    foods: List[Food] #foods offered by the restaurant that is list of class Food

# Create Food instance within the foods list

restaurant_instance = Restaurant( #object of the restaurant class
    name='Tasty Bites',
    location='123, Flavor Street',
    foods=[
        Food(
            name='Cheese Burger',
            price=12.50,
            ingredients=['Cheese', 'Tomato Sauce']
        )
    ]
)
display(restaurant_instance.foods[0].name)
display(restaurant_instance.model_dump_json())

'Cheese Burger'

'{"name":"Tasty Bites","location":"123, Flavor Street","foods":[{"name":"Cheese Burger","price":12.5,"ingredients":["Cheese","Tomato Sauce"]}]}'